In [1]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/7

In [2]:
import os

from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search

from google.genai import types  # Para criar conteúdos (Content e Part)

import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

import requests # Para fazer requisições HTTP

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [4]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [35]:
##########################################
# --- Agente 1: Mentor QA --- #
##########################################
def agente_mentor_qa(topico):

    mentor_qa = Agent(
        name="agente_mentor_qa",
        model="gemini-2.0-flash",
        instruction="""
        Você é um profissional especialista em qualidade de software e testes, com a capacidade
        de mentorar quaisquer pessoas, tanto as que atuam nessa área (novatos ou veteranos) quanto àquelas que estão em transição.

        Seus conhecimentos são baseados:

        - Na literatura dos autores de maior relevância na área de Qualidade de Software e Testes como,
        Roger Pressman, Robert C. Martin (Uncle Bob), Martin Fowler, Kent Beck, Elisabeth Hendrickson, Joseph Juran,
        Philip Crosby, Edwards Deming, Armand Feigenbaum, Kaoru Ishikawa e Ian Sommerville.

        - Em conteúdos dos materiais do ISTQB/BSTQB (Syllabus) para os exames de certificação como,
        CTFL, CTAL e demais certificações relevantes para área de qualidade de software e testes.

        - Em documentações oficiais de ferramentas utilizadas na realização dos testes, por exemplo, ferramentas de automação web (Cypress, Selenium, Playwright, Robotframework, etc),
        ferramentas de teste de API (Postman, Insomnia, etc), ferramentas para gestão dos testes e dos bugs (Jira, TestLink, Bugzilla, Qase, etc), ferramentas de testes de performance (Jmeter, K6, Locust, etc), etc.

        Adote a ferramenta do Google (google_search) a seu favor para ter acesso à informações relevantes sobre a área de qualidade de software e testes para te ajudar a engrandecer
        sua base de conhecimento com informações valiosas e úteis.

        Caso o usuário realize perguntas quem são totalmente fora do contexto do qual você é especialista, você tem a liberdade de interromper o assunto de forma totalmente cortês pois, seu foco não é esse, visto que você
        é um especialista em qualidade de software e testes. Busque orientar o usuário sobre seu propósito para que ele evite questionar sobre questões fora do seu contexto.
        """,
        description="Agente que ajuda em qualquer assunto dentro da área de QA",
        tools=[google_search]
    )

    entrada_do_mentor_qa = f"Tópico: {topico}"

    mentoria = call_agent(mentor_qa, entrada_do_mentor_qa)
    return mentoria

In [49]:
#####################################################
# --- Agente 2: Especialista em Plano de Testes --- #
#####################################################
def agente_expert_plano_de_testes(contexto):
  expert_plano_de_testes = Agent(
        name="agente_expert_plano_de_testes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um expert em elaborar e estruturar planos de testes impecáveis para qualquer projeto de software.
        Seja didático na sua explicação do plano de testes, principalmente para aqueles usuários que você notar que não apresentam tanta familiaridade na
        elaboração de um plano de testes.
        Se necessário, você pode fornecer um modelo-exemplo de um plano de testes baseado na proposta de software que o usuário vier a informar.
        Quando o usuário te fornecer um exemplo de plano de testes, avalie com atenção, forneça feedbacks construtivos e se for cabível, forneça dicas para melhorias.

        Seu contexto é restrito a tratar sobre "Plano de Testes", tudo que for fora desse contexto é proibido e você deve deixar claro ao usuário sobre seu propósito.
        """,
        description="Agente que ajuda com Plano de Testes",
        tools=[google_search]
  )
  entrada_do_expert_plano_de_testes = f"Contexto: {contexto}"
  plano = call_agent(expert_plano_de_testes, entrada_do_expert_plano_de_testes)
  return plano

In [48]:
###############################################################
# --- Agente 3: Especialista em Cenários e Casos de Teste --- #
###############################################################
def agente_expert_cenarios_casos_de_testes(contexto):
  expert_cenarios_casos_de_testes= Agent(
        name="agente_expert_cenarios_casos_de_testes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um expert em elaborar e estruturar Cenários e Casos de Teste impecáveis para qualquer projeto de software baseado nos seus requisitos.
        Seja didático na sua explicação do Cenários e Casos de Teste, principalmente para aqueles usuários que você notar que não apresentam tanta familiaridade na elaboração.
        Se necessário, você pode fornecer um modelo-exemplo de Cenários e Casos de Teste baseado na proposta do software que o usuário vier a informar.
        Quando o usuário te fornecer um exemplo de Cenários e Casos de Teste, avalie com atenção, forneça feedbacks construtivos e se for cabível, forneça dicas para melhorias.
        Normalmente, este documento de Cenários de Casos de Teste pode apresentar os seguintes campos:
        - ID
        - Descrição
        - Prioridade
        - Tipo de Teste
        - Pré-condições
        - Pós-condições
        - Critérios de aceitação
        - Passo a Passo (geralmente, usando linguagem Gherkin)
        - Resultado Esperado
        - Resultado Obitdo
        - Dados de Teste
        - Status
        - Evidências

        Seu contexto é restrito a tratar sobre "Cenários e Casos de Teste", tudo que for fora desse contexto é proibido e você deve deixar claro ao usuário sobre seu propósito.
        """,
        description="Agente que ajuda com Cenários e Casos de Teste",
        tools=[google_search]
  )
  entrada_do_expert_cenarios_casos_de_testes = f"Contexto: {contexto}"
  cct = call_agent(expert_cenarios_casos_de_testes, entrada_do_expert_cenarios_casos_de_testes)
  return cct

In [54]:
#######################################################
# --- Agente 4: Especialista em Relatório de Bugs --- #
#######################################################
def agente_expert_relatório_bugs(contexto):
  expert_relatório_bugs= Agent(
        name="agente_expert_relatório_bugs",
        model="gemini-2.0-flash",
        instruction="""
        Você é um expert em elaborar e estruturar Relatórios de Bugs impecáveis para qualquer projeto de software que passou pelo processo de testes, onde houve de bugs.
        Seja didático na sua explicação de Relatório de Bugs, principalmente para aqueles usuários que você notar que não apresentam tanta familiaridade na
        elaboração.
        Você pode fornecer um modelo exemplo de Relatórios de Bugs baseado nas informações fornecidas (bugs identificados) pelo usuário.
        Se o usuário te fornecer um exemplo de Relatórios de Bugs, avalie com atenção, forneça feedbacks construtivos e se for cabível, forneça dicas para melhorias.
        Para seu conhecimento, normalmente, este documento de Relatórios de Bugs pode apresentar os seguintes campos:
        - ID
        - Descrição
        - Severidade do Bug
        - Prioridade de Correção
        - Status
        - Passo a Passo para simular o bug
        - Comportamento Esperado
        - Comportamento Obtido
        - Ambiente
        - Funcionalidade Afetada
        - Caso de Teste Relacionado
        - Evidências

        Seu contexto é restrito a tratar sobre "Relatório de Bugs", tudo que for fora desse contexto é proibido e você deve deixar claro ao usuário sobre seu propósito.
        """,
        description="Agente que ajuda com Relatório de Bugs",
        tools=[google_search]
  )
  entrada_do_expert_relatório_bugs = f"Contexto: {contexto}"
  r_bugs = call_agent(expert_relatório_bugs, entrada_do_expert_relatório_bugs)
  return r_bugs

In [55]:
##########################################################
# --- Agente 5: Especialista em Resultado dos Testes --- #
##########################################################
def agente_expert_resultado_testes(contexto):
  expert_resultado_testes= Agent(
        name="agente_expert_resultado_testes",
        model="gemini-2.0-flash",
        instruction="""
        Você é um expert em elaborar e estruturar Resultados dos Testes impecáveis para qualquer projeto de software que passou pelo processo de testes.
        Seja didático na sua explicação dos resultados dos testes, principalmente para aqueles usuários que você notar que não apresentam tanta familiaridade na elaboração.
        Você pode fornecer um modelo exemplo de resultados dos testes ajudar no entendimento do usuário.

        Seu contexto é restrito a tratar sobre "Resultado dos Testes", tudo que for fora desse contexto é proibido e você deve deixar claro ao usuário sobre seu propósito.
        """,
        description="Agente que ajuda com os Resultados dos Testes",
        tools=[google_search]
  )
  entrada_do_expert_resultado_testes = f"Contexto: {contexto}"
  r_testes = call_agent(expert_resultado_testes, entrada_do_expert_resultado_testes)
  return r_testes

In [ ]:

print("#################################################")
print("######### 🚀 Iniciando o OneQway... 🚀 #########")
print("#################################################")
print("\n")
print("🤖 Olá, humano(a)! Eu meu chamo OneQway, sou seu mestre digital de QA.\n")
print("🤖 Para te guiar da melhor forma possível, listo as seguintes opções:")
print("1️⃣ para aprender sobre qualquer assunto dentro da área de QA.")
print("2️⃣ para conversar especificamente da elaboração de um Plano de Testes.")
print("3️⃣ para conversar especificamente da elaboração dos Cenários e Casos de Teste.")
print("4️⃣ para conversar especificamente da elaboração de Relátorio de Bugs.")
print("5️⃣ para conversar especificamente da elaboração do Resultado dos Testes.")
print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.")

opcao_valida = False

while not opcao_valida:  # Enquanto a opção NÃO for válida...
    opcao = input("\n🤖 Agora me forneça uma opção válida: ")

    # MENTOR QA ################################################
    if opcao == "1":
        opcao_valida = True  # Agora a opção é válida, podemos sair do while
        print("\n🚀 Legal! Vamos aprender sobre QA!")
        while True: # Vamos criar um loop aqui para garantir que o tópico seja fornecido
            topico = input("\n🤖 O que você gostaria de aprender hoje dentro do universo QA❓\n")
            if topico: # Verifica se o tópico não está vazio
                print(f"\n🤖 Maravilha! Um momento, estou verificando minha base de conhecimento sobre: {topico}.\n")
                base_de_conhecimento = agente_mentor_qa(topico)
                print("📝 Resultado Obtido da Base de Conhecimento QA")
                display(to_markdown(base_de_conhecimento))
                print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                while True: # Loop para conversas subsequentes sobre o mesmo tópico
                    topico = input("🤖 O que gostaria de aprender agora❓\n")
                    if topico.lower() == 'fim':
                        print('\n🤖 Até a próxima, humano(a)❗')
                        break # Sai do loop de conversas subsequentes
                    else:
                        base_de_conhecimento = agente_mentor_qa(topico)
                        display(to_markdown(base_de_conhecimento))
                        print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                break # Sai do loop de solicitação inicial de tópico (já que obtivemos um)
            else:
                print("\n🤖 Preciso que me informe o que você gostaria de aprender hoje❗\n")
                print("#################################################")
                # O loop 'while True' vai continuar aqui até que um tópico seja digitado



    # PLANO DE TESTES ################################################
    elif opcao == "2":
        print("\n📝 Ótimo! Vamos falar sobre Plano de Testes.")
        opcao_valida = True
        while True: # Novo loop para garantir que o contexto seja fornecido
            contexto = input("\n🤖 O que você gostaria de saber sobre Plano de Testes❓\n")
            if contexto: # Verifica se o contexto não está vazio
                print(f"\n🤖 Maravilha! Um momento, estou analisando seu contexto: {contexto}.\n")
                contexto_plano = agente_expert_plano_de_testes(contexto)
                print("📝 Resultado Obtido ")
                display(to_markdown(contexto_plano))
                print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                while True: # Loop para conversas subsequentes sobre o plano de testes
                    contexto = input("🤖 O que gostaria de saber agora sobre Plano de Testes❓\n")
                    if contexto.lower() == 'fim':
                        print('\n🤖 Até a próxima, humano(a)❗')
                        break # Sai do loop de conversas subsequentes
                    else:
                        contexto_plano = agente_expert_plano_de_testes(contexto)
                        display(to_markdown(contexto_plano))
                        print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                break # Sai do loop de solicitação inicial de contexto (já que obtivemos um)
            else:
                print("\n🤖 Preciso que me informe o que você gostaria de saber sobre Plano de Testes❗\n")
                print("#################################################")
                # O loop 'while True' vai continuar aqui até que um contexto seja digitado


    # CENÁRIOS E CASOS DE TESTE ################################################
    elif opcao == "3":
      print("\n✍️ Ótimo! Vamos falar sobre Cenários e Casos de Teste.")
      opcao_valida = True
      while True: # Novo loop para garantir que o contexto seja fornecido
            contexto = input("\n🤖 O que você gostaria de saber sobre Cenários e Casos de Teste❓\n")
            if contexto: # Verifica se o contexto não está vazio
                print(f"\n🤖 Maravilha! Um momento, estou analisando seu contexto: {contexto}.\n")
                contexto_cct = agente_expert_cenarios_casos_de_testes(contexto)
                print("📝 Resultado Obtido ")
                display(to_markdown(contexto_cct))
                print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                while True: # Loop para conversas subsequentes sobre o Cenários e Casos de Teste
                    contexto = input("🤖 O que gostaria de saber agora sobre Cenários e Casos de Teste❓\n")
                    if contexto.lower() == 'fim':
                        print('\n🤖 Até a próxima, humano(a)❗')
                        break # Sai do loop de conversas subsequentes
                    else:
                        contexto_cct = agente_expert_cenarios_casos_de_testes(contexto)
                        display(to_markdown(contexto_cct))
                        print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                break # Sai do loop de solicitação inicial de contexto (já que obtivemos um)
            else:
                print("\n🤖 Preciso que me informe o que você gostaria de saber sobre Cenários e Casos de Teste❗\n")
                print("#################################################")
                # O loop 'while True' vai continuar aqui até que um contexto seja digitado

    # RELATÓRIO DE BUGS ################################################
    elif opcao == "4":
      print("\n🐞 Ótimo! Vamos falar sobre Relátorio de Bugs.")
      opcao_valida = True
      while True: # Novo loop para garantir que o contexto seja fornecido
            contexto = input("\n🤖 O que você gostaria de saber sobre Relátorio de Bugs❓\n")
            if contexto: # Verifica se o contexto não está vazio
                print(f"\n🤖 Maravilha! Um momento, estou analisando seu contexto: {contexto}.\n")
                contexto_r_bugs = agente_expert_relatório_bugs(contexto)
                print("📝 Resultado Obtido ")
                display(to_markdown(contexto_r_bugs))
                print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                while True: # Loop para conversas subsequentes sobre o Relátorio de Bugs
                    contexto = input("🤖 O que gostaria de saber agora sobre Relátorio de Bugs❓\n")
                    if contexto.lower() == 'fim':
                        print('\n🤖 Até a próxima, humano(a)❗')
                        break # Sai do loop de conversas subsequentes
                    else:
                        contexto_r_bugs = agente_expert_relatório_bugs(contexto)
                        display(to_markdown(contexto_r_bugs))
                        print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                break # Sai do loop de solicitação inicial de contexto (já que obtivemos um)
            else:
                print("\n🤖 Preciso que me informe o que você gostaria de saber sobre Relátorio de Bugs❗\n")
                print("#################################################")
                # O loop 'while True' vai continuar aqui até que um contexto seja digitado

    # RESULTADOS DOS TESTES ################################################
    elif opcao == "5":
      print("\n📊 Ótimo! Vamos falar sobre Resultados dos Testes.")
      opcao_valida = True
      while True: # Novo loop para garantir que o contexto seja fornecido
            contexto = input("\n🤖 O que você gostaria de saber sobre Resultado dos Testes❓\n")
            if contexto: # Verifica se o contexto não está vazio
                print(f"\n🤖 Maravilha! Um momento, estou analisando seu contexto: {contexto}.\n")
                contexto_r_testes = agente_expert_resultado_testes(contexto)
                print("📝 Resultado Obtido ")
                display(to_markdown(contexto_r_testes))
                print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                while True: # Loop para conversas subsequentes sobre o Resultado dos Testes
                    contexto = input("🤖 O que gostaria de saber agora sobre Resultado dos Testes❓\n")
                    if contexto.lower() == 'fim':
                        print('\n🤖 Até a próxima, humano(a)❗')
                        break # Sai do loop de conversas subsequentes
                    else:
                        contexto_r_testes = agente_expert_resultado_testes(contexto)
                        display(to_markdown(contexto_r_testes))
                        print("\n🤖 Caso deseje encerrar nossa conversa, basta digitar 'FIM'.\n")
                break # Sai do loop de solicitação inicial de contexto (já que obtivemos um)
            else:
                print("\n🤖 Preciso que me informe o que você gostaria de saber sobre Resultado dos Testes❗\n")
                print("#################################################")
                # O loop 'while True' vai continuar aqui até que um contexto seja digitado

    elif opcao.lower() == "fim":
      print("\n🤖 Até a próxima, humano(a)❗")
      opcao_valida = True

    else:
      print("\n🤔 Hmm, essa não é uma opção válida. Tente de novo!")